In [1]:
from os.path import exists

import pandas as pd
import numpy as np

import statsmodels.api as sm


import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from scipy.io.wavfile import write
from scipy.signal import butter, lfilter, freqz
from scipy.fftpack import fft,rfft, rfftfreq

## Observational data comes from SilSo (http://www.sidc.be/silso/dayssnplot)

SN_d_tot_V2.0.csv<br>
SN_ms_tot_V2.0.csv

## Cleaning the data, if needed

In [3]:
if exists('../data/pickled_sunspot_data.pkl'):
    print('Using the data that has already been cleaned')
else:
    print("Clean data isn't here. Cleaning and exporting")
    
    sun = pd.read_csv('../data/SN_d_tot_V2.0.csv', sep = ';')
    sun.columns = ['Year', 'Month', 'Day', 'Fractional Date', 'SunSpot Count', "DailyStDev", "Observations", 'Indicator']
    sun['Date'] = sun['Year'].map(str)+ '-' + sun['Month'].map(str) + '-' + sun['Day'].map(str)
    pd.to_datetime(sun['Date'], utc=False)
    
    sun = sun[['Date', 'SunSpot Count', 'DailyStDev', 'Observations', 'Indicator', 'Fractional Date'] ]
    sun['Indicator'] = sun['Indicator'].astype(bool)
    
    sun[['Observations', 'SunSpot Count']] = sun[['Observations', 'SunSpot Count']].astype(int)
    sun.to_pickle('../data/pickled_sunspot_data.pkl')

Using the data that has already been cleaned


In [4]:
sun = pd.read_pickle('../data/pickled_sunspot_data.pkl')
sun.head()

,Date,SunSpot Count,DailyStDev,Observations,Indicator,Fractional Date
0,1818-1-2,-1,-1.0,0,True,1818.004
1,1818-1-3,-1,-1.0,0,True,1818.007
2,1818-1-4,-1,-1.0,0,True,1818.010
3,1818-1-5,-1,-1.0,0,True,1818.012
4,1818-1-6,-1,-1.0,0,True,1818.015


## Initial plots

This plot below seems a little supicious. I haven't summed up any of the data but have what appears to be summations. Also, I'm not clear on what the part in white would be.

In [ ]:
plt.figure(figsize = (20, 5))
ax = plt.gca()

sun.plot(kind='line', x='Date', y='SunSpot Count', ax=ax)
plt.show()

In [ ]:
sun.describe()

In [ ]:
len(sun)

Hunh. Well, that would seem to dispel my first concern. 

This is a good place to stop for the night. I think I want to look about casting the indicator as a bool and Observations and total SunSpots as integers. That should be faster, take less memory, and should be a bit clearer.


In [ ]:
sun['Indicator'] = sun['Indicator'].astype(bool)
sun[['Observations', 'SunSpot Count']] = sun[['Observations', 'SunSpot Count']].astype(int)

In [ ]:
window = 730 #makes for a two year rolling average
sun['rolling'] = sun.iloc[:,1].rolling(window).mean()

In [ ]:
plt.figure(figsize = (20, 5))
ax = plt.gca()

sun.plot(kind='line', x='Date', y='SunSpot Count', ax=ax)
sun.plot(kind='line', x='Date', y='rolling', ax=ax)

plt.show()

In [ ]:
data = sun[1:len(sun)]

mat

In [ ]:
def get_fft_values(y_values, T, N, f_s):
    f_values = np.linspace(0.0, 1.0/(2.0*T), N//2)
    fft_values_ = fft(y_values)
    fft_values = 2.0/N * np.abs(fft_values_[0:N//2])
    return f_values, fft_values

In [ ]:
t_n = 200
N = 200 #sample rate (in Hz) must equal last of x=np.linspace(0,10,300) below
T = t_n / N
f_s = 1/T #freq

x_value = np.linspace(0,t_n,N)

#dummy values
amplitudes = [1, 1, 1]
frequencies = [30, 10, 1]

y_values = [amplitudes[i]*np.sin(2*np.pi*frequencies[i]*x_value) for i in range(0,len(amplitudes))]
composite_y_value = np.sum(y_values, axis=0)

f_values, fft_values = get_fft_values(composite_y_value, T, N, f_s)

colors = ['k', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b']

In [ ]:
f_values, fft_values = get_fft_values(composite_y_value, T, N, f_s)
 
plt.plot(f_values, fft_values, linestyle='-', color='blue')
plt.xlabel('Frequency [Hz]', fontsize=16)
plt.ylabel('Amplitude', fontsize=16)
plt.title("Frequency domain of the signal", fontsize=16)
plt.show()

In [ ]:
t_n = 200
N = 200 #sample rate (in Hz) must equal last of x=np.linspace(0,10,300) below
T = t_n / N
f_s = 1/T #freq

x_value = np.linspace(0,t_n,N)

In [ ]:
scipy.fft.fft(sun['SunSpot Count'], n=None, axis=- 1, norm=None, overwrite_x=False, workers=None, plan=None)

It's been a while so a refresher on how FFT works would be useful

In [ ]:
SAMPLE_RATE = 44100  # Hertz
DURATION = 5  # Seconds

def generate_sine_wave(freq, sample_rate, duration):
    x = np.linspace(0, duration, sample_rate * duration, endpoint=False)
    frequencies = x * freq
    # 2pi because np.sin takes radians
    y = np.sin((2 * np.pi) * frequencies)
    return x, y


# Number of samples in normalized_tone
N = SAMPLE_RATE * DURATION


# Generate a 2 hertz sine wave that lasts for 5 seconds
x, y = generate_sine_wave(2, SAMPLE_RATE, DURATION)
plt.plot(x, y)
plt.show()

In [ ]:
_, nice_tone = generate_sine_wave(400, SAMPLE_RATE, DURATION)
_, noise_tone = generate_sine_wave(4000, SAMPLE_RATE, DURATION)
noise_tone = noise_tone * 0.3

mixed_tone = nice_tone + noise_tone


In [ ]:
normalized_tone = np.int16((mixed_tone / mixed_tone.max()) * 32767)

plt.plot(normalized_tone[:1000])
plt.show()

In [ ]:
# Remember SAMPLE_RATE = 44100 Hz is our playback rate
write("mysinewave.wav", SAMPLE_RATE, normalized_tone)


In [ ]:
# Note the extra 'r' at the front
yf = rfft(normalized_tone)
xf = rfftfreq(N, 1 / SAMPLE_RATE)

plt.plot(xf, np.abs(yf))


plt.show()


In [ ]:
yf.size

### Let's take a step back an use find_peaks to see if I can get that to lablel the values.
### I'm using the find_peaks and using the electrocardiogram dataset 

In [ ]:
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks
import numpy as np


In [ ]:
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks
plt.figure(figsize = (20, 5))

x = electrocardiogram()[2000:4000]

peaks, _ = find_peaks(x, distance=150)

c= np.diff(peaks)

plt.plot(x)
plt.plot(peaks, x[peaks], 'x')
plt.show()
c

In [ ]:
plt.figure(figsize = (20, 5))
x = electrocardiogram()[2000:4000]
peaks, _ = find_peaks(x, height=0, distance=100)
plt.plot(x)
plt.plot(peaks, x[peaks], "x")
plt.plot(np.zeros_like(x), "--", color="gray")
plt.show()

In [ ]:
p, q = peaks, x[peaks]

peaks, _ = find_peaks(x, distance=150)
np.diff(peaks)


In [ ]:
fig, ax = plt.subplots()
ax.scatter(p, q)
plt.gcf().set_size_inches(20, 10)
for i, txt in enumerate(p):
    ax.annotate(txt, (p[i], q[i]))


plt.plot(x)
plt.plot(peaks, x[peaks], 'x')

plt.show()

In [ ]:
fft = np.fft.fft(sun['SunSpot Count'])

for i in range(2):
    print("Value at index {}:\t{}".format(i, fft[i + 1]), "\nValue at index {}:\t{}".format(fft.size -1 - i, fft[-1 - i]))

### Next step --

 1. Take the numbers from `cell 29` and adapt the code to display the frequencies in `cell 24` and `cell 17`
 2. That should allow me to see the frequencies in the test case. Might need a periodic filter to take out the non-peaks
 3. That souuld give me a start on finding the freqs for the `sun` dataset